In [11]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import math 
import numpy as np 
import matplotlib.pyplot as plt
import logging 
import cv2 as cv

datasets, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True) # Loads the data
train_dataset, test_dataset = datasets['train'], datasets['test'] # Splits the dataset 


In [12]:
class_name = metadata.features['label'].names # Prints the names of the labels
print(class_name)
print(metadata.features) # Prints the features


num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), #flattens the 3d image into one array of numbers
    tf.keras.layers.Dense(512, activation=tf.nn.relu), #creates 128 level of neural networks
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) #softmax produces probability distribution 
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy']) 

BATCH_SIZE = 32 #Batch size is the number of iterations the neural network does before it is updated 
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE) #Updates dataset with batch included
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
})
Number of training examples: 60000
Number of test examples:     10000


In [13]:
import os
checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_dataset, epochs=10, steps_per_epoch=math.ceil(num_test_examples/32), callbacks=[cp_callback])

model.save_weights("model_weights")
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)
print('Loss: ', test_loss)

for test_images, test_labels in test_dataset.take(1): #predict one sample from the data
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

predictions.shape
print(np.argmax(predictions[0]))

Epoch 1/10
311/313 [============================>.] - ETA: 0s - loss: 4.6921 - accuracy: 0.7127
Epoch 1: saving model to training_2\cp.ckpt
313/313 [==============================] - 7s 14ms/step - loss: 4.6708 - accuracy: 0.7128
Epoch 2/10
310/313 [============================>.] - ETA: 0s - loss: 0.8691 - accuracy: 0.7664
Epoch 2: saving model to training_2\cp.ckpt
313/313 [==============================] - 4s 14ms/step - loss: 0.8675 - accuracy: 0.7662
Epoch 3/10
313/313 [==============================] - ETA: 0s - loss: 0.6689 - accuracy: 0.7903
Epoch 3: saving model to training_2\cp.ckpt
313/313 [==============================] - 4s 14ms/step - loss: 0.6689 - accuracy: 0.7903
Epoch 4/10
312/313 [============================>.] - ETA: 0s - loss: 0.5762 - accuracy: 0.8081
Epoch 4: saving model to training_2\cp.ckpt
313/313 [==============================] - 4s 14ms/step - loss: 0.5759 - accuracy: 0.8080
Epoch 5/10
311/313 [============================>.] - ETA: 0s - loss: 0.5335 - a

In [25]:
import matplotlib.pyplot as plt
for test_images, test_labels in test_dataset.take(3): #predict one sample from the data
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  # print(type(test_images), (test_labels))
  # print(test_labels)
  # plt.imshow(test_images)
  predictions = model.predict(test_images)

predictions.shape
print(predictions)
print(np.argmax(predictions[0]))


1/1 [==============================] - 0s 39ms/step
[[1.22523670e-05 1.55056156e-07 1.92017779e-01 3.33811295e-05
  8.07420313e-01 1.18348620e-08 5.14241692e-04 1.26773714e-09
  1.87289845e-06 3.71303793e-10]
 [6.93838656e-01 8.33966420e-04 1.40907206e-02 2.31047738e-02
  4.40708362e-03 2.04402080e-04 2.61413336e-01 1.19777927e-04
  1.94419979e-03 4.31628432e-05]
 [5.01720118e-04 3.18449056e-05 6.87812626e-01 1.50179223e-03
  3.00302327e-01 3.60631384e-06 9.82900802e-03 1.61783623e-06
  1.54843965e-05 1.82601596e-08]
 [2.05344870e-04 2.31395532e-07 1.55342162e-01 5.78116626e-04
  7.97043920e-01 5.86962460e-06 4.67895940e-02 1.66062350e-07
  3.45452427e-05 1.51823670e-07]
 [1.18436707e-04 7.87085014e-07 5.23225553e-02 2.06580735e-03
  5.84135473e-01 1.18945791e-05 3.61330450e-01 2.33992205e-06
  1.11464524e-05 1.17829131e-06]
 [4.38045930e-07 7.90240706e-10 1.04090236e-06 2.16013540e-09
  2.70014516e-06 9.99951839e-01 9.98044669e-08 4.27755440e-05
  3.46409195e-07 7.58577244e-07]
 [3.35